# install sci-kit learn and restart runtime

In [70]:
!pip install scikit-learn==1.2.2
!pip install imblearn

#setup:
- connect to google drive in this step

In [71]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow
import random

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
project_path = '/content/drive/MyDrive/230702/demo-me-2021-07-14'

# Load classifier model:
requires classifier file name

In [79]:
import pickle

classifier_name = input('Enter the classifier name: ')
print(f'You entered {classifier_name}')

# open a file, where you stored the pickled data
file = open(project_path+ '/classifiers/' + classifier_name+ '.pickle', 'rb')

# dump information to that file
RFC = pickle.load(file)
print(RFC)

# close the file
file.close()

Enter the classifier name: 230704_6moredata
You entered 230704_6moredata
RandomForestClassifier(n_estimators=10, random_state=5)


# Get video features:

In [75]:
# load an all_i_features pickle
video_name = '221127_PZ90_1'
with open(project_path + "/behaviors/" + video_name + "_features.pickle", 'rb') as file:
    all_i_features = pickle.load(file)

# Forest features data:

In [76]:
input_features = all_i_features
print(input_features.shape)

(36000, 8)


In [77]:
print(np.any(np.isnan(input_features)))
print(np.any(np.isinf(input_features)))

False
False


In [78]:
input_features[input_features < -10000] = -10000 # should all be positive though
input_features[input_features > 10000] = 10000

#Run classifier smote RFC

In [80]:
#Importing essential libraries
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.ensemble import RandomForestClassifier

In [81]:
y_pred = RFC.predict(input_features)

#turn y_pred into timestamps of changes

# get labeled behavior video

In [82]:
import cv2
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture(project_path + '/videos/' + video_name + 'DLC_resnet50_demoJul14shuffle1_50000_full.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)

fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
out = cv2.VideoWriter(project_path + '/'+video_name+'_out.mp4',fourcc, 30, size)
#out = cv2.VideoWriter(project_path + '/behaviors/' + 'video.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

i = 0;
max_frames = 36000
while(True):
  if i >= max_frames:
    break
  ret, frame = cap.read()

  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(frame,
              str(i),
              (150, 20),
              font, 0.5,
              (0, 0, 0),
              2,
              cv2.LINE_4)

  text = 'none'
  label = y_pred[i]
  if label == 1:
    text = 'Mount'
    cv2.putText(frame,
              text,
              (20, 20),
              font, 1,
              (0, 255, 0.75),
              2,
              cv2.LINE_4)
  elif label == 0:
    text = 'Nonmount'
    cv2.putText(frame,
                text,
                (20, 20),
                font, 0.75,
                (0, 0, 0),
                2,
                cv2.LINE_4)
  out.write(frame)
  i = i+1

  # creating 'q' as the quit
  # button for the video
  if cv2.waitKey(1) & 0xFF == ord('q'):
      break

cap.release()

out.release()
# close all windows
cv2.destroyAllWindows()